In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.preprocessing import StandardScaler , MaxAbsScaler , MinMaxScaler 
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression , LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
import warnings
import scipy
from sklearn.preprocessing import OrdinalEncoder
encoder=OrdinalEncoder()

In [54]:
df=pd.read_csv('auto_train.csv')

In [55]:
df = df.drop(df[df['Price'] == 'Պայմ.'].index)
df.drop(4990, inplace=True)

# PRICE 

In [4]:
dollar_indexes=df.index[df['Price'].str.contains('\֏')]
euro_indexes=df.index[df['Price'].str.contains('\€')]
df['Price'] = df['Price'].str.replace('$', '')
df['Price'] = df['Price'].str.replace('֏', '')
df['Price'] = df['Price'].str.replace('€', '')
df['Price'] = df['Price'].str.replace(' ', '')
df['Price'] = df['Price'].astype('int64')
dram_to_tollar=385.88
for i in dollar_indexes:
    df.loc[i,'Price']= df.loc[i,'Price']/dram_to_tollar
dram_to_euro=420.61
for m in euro_indexes:
    df.loc[m,'Price']= df.loc[m,'Price']/dram_to_euro

#  Mileage

In [5]:
def improve_Milage(x , mile=scipy.constants.mile/1000):
    x = x.strip()
    if "մղոն" in x:
        x = x.strip("մղոն")
        return int(x.strip())*mile
    elif "կմ" in x:
        x = x.strip("կմ")
        return int(x.strip())
    else:
        return np.nan

In [6]:
df["Mileage"] = df["Mileage"].apply(improve_Milage)

# Model

In [7]:
df['Model'] = df['Model'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [8]:
ind = df[df["Mileage"].isna()].index

In [9]:
df.loc[ind , "Mileage" ]  = df[df["Mileage"].isna()]["Model"].values

In [10]:
df.loc[ind , 'Model'] = np.nan

In [11]:
df["Mileage"] = df["Mileage"].apply(lambda x : improve_Milage(x) if isinstance(x , str) else x)

In [12]:
# Filling missing models
df.loc[[3495,314,1558,1232,1598],'Model']='Accent'
df.loc[[4254,2416,5223,5290,3503,815,5723],'Model']='Elantra'
df.loc[[3968,1756,1595,3886,4257,3596],'Model']='Sonata'
df.loc[[1872,3125,2989,3681],'Model']='Santa Fe'
df.loc[[1010],'Model']='Tucson'
df.loc[[3670],'Model']='Palisade'
df.loc[[5661],'Model']='Forte'
df['Body style'] = df['Body style'].fillna('Սեդան')

#  Improving shufled columns

In [13]:
df.loc[ind , "Custom cleared" ]  = df[df["Custom cleared"].isna()]["Color"].values
df.loc[ind , 'Color'] = np.nan
df.loc[ind ,"Color"]  = df[df["Color"].isna()]["Engine"].values
df.loc[ind , 'Engine'] = np.nan
df.loc[ind , "Engine" ]  = df[df["Engine"].isna()]["Hand drive"].values
df.loc[ind , 'Hand drive'] = np.nan
df.loc[ind , "Hand drive" ]  = df[df["Hand drive"].isna()]["Gearbox"].values
df.loc[ind , 'Gearbox'] = np.nan
df.loc[ind , "Gearbox" ]  = df[df["Gearbox"].isna()]["Body style"].values
df.loc[ind , 'Body style'] = np.nan

# Body Style

In [14]:
df['Body style'] = df['Body style'].apply(lambda x: x.strip() if isinstance(x, str) else x)
value_counts = df['Body style'].value_counts()
values_to_replace = value_counts[value_counts < 7].index
df.loc[df['Body style'].isin(values_to_replace), 'Body style'] = np.nan

In [15]:
encoded_column = encoder.fit_transform(df[['Body style']])
df['Body style'] = encoded_column

In [16]:
nan_index=df[df['Body style'].isna()].index
pred_body=df.loc[:,['Year','Make','Model','Body style']]

In [17]:
encoded_column = encoder.fit_transform(pred_body[['Make']])
pred_body['Make'] = encoded_column
encoded_column = encoder.fit_transform(pred_body[['Model']])
pred_body['Model'] = encoded_column

In [18]:
test_body=pred_body.loc[nan_index,['Year','Make','Model']]
train_body=pred_body.drop(nan_index)

In [19]:
from sklearn.ensemble import RandomForestClassifier
log_model = RandomForestClassifier()
log_model.fit(train_body.drop('Body style',axis=1),train_body['Body style'])
b=log_model.predict(test_body)

In [20]:
df.loc[nan_index , 'Gearbox'] = b

# Improve Gearbox

In [21]:
df['Gearbox'] = df['Gearbox'].apply(lambda x: x.strip() if isinstance(x, str) else x)
value_counts = df['Gearbox'].value_counts()
values_to_replace = value_counts[value_counts < 12].index
df.loc[df['Gearbox'].isin(values_to_replace), 'Gearbox'] = np.nan

In [22]:
def transform_value(value):
    if value == 'Ավտոմատ':
        return 3
    elif value == 'Մեխանիկական':
        return 1
    elif value == 'Վարիատոր':
        return 2
    else:
        return np.nan
df['Gearbox'] = df['Gearbox'].apply(transform_value)

In [23]:
nan_index=df[df['Gearbox'].isna()].index
new=df
# new_df = df.drop(nan_index)
pred_gear=new.loc[:,['Year','Make','Model','Gearbox']]

In [24]:
encoded_column = encoder.fit_transform(pred_gear[['Make']])
pred_gear['Make'] = encoded_column
encoded_column = encoder.fit_transform(pred_gear[['Model']])
pred_gear['Model'] = encoded_column

In [25]:
test_gearbox=pred_gear.loc[nan_index,['Year','Make','Model']]
train_gearbox=pred_gear.drop(nan_index)

In [26]:
from sklearn.ensemble import RandomForestClassifier
log_model = RandomForestClassifier()
log_model.fit(train_gearbox.drop('Gearbox',axis=1),train_gearbox['Gearbox'])
z=log_model.predict(test_gearbox)

In [27]:
df.loc[nan_index , 'Gearbox'] = z

# Hand drive

In [28]:
df['Hand drive'] = df['Hand drive'].apply(lambda x: x.strip() if isinstance(x, str) else x)
body_style_counts = df['Hand drive'].value_counts()
valuess_to_replace = body_style_counts[body_style_counts < 28].index
df.loc[df['Hand drive'].isin(valuess_to_replace), 'Hand drive'] = np.nan

In [29]:
def transform_valuee(value):
    if value == 'Ձախ':
        return 2
    elif value == 'Աջ':
        return 1
    else:
        return np.nan
df['Hand drive'] = df['Hand drive'].apply(transform_valuee)

In [30]:
nan_index=df[df['Hand drive'].isna()].index
pred_hand=df.loc[:,['Year','Make','Model','Hand drive']]

In [31]:
encoded_column = encoder.fit_transform(pred_hand[['Make']])
pred_hand['Make'] = encoded_column
encoded_column = encoder.fit_transform(pred_hand[['Model']])
pred_hand['Model'] = encoded_column

In [32]:
test_hand=pred_hand.loc[nan_index,['Year','Make','Model']]
train_hand=pred_hand.drop(nan_index)

In [33]:
from sklearn.ensemble import RandomForestClassifier
log_model = RandomForestClassifier()
log_model.fit(train_hand.drop('Hand drive',axis=1),train_hand['Hand drive'])
k=log_model.predict(test_hand)

In [34]:
df.loc[nan_index , 'Gearbox'] = k

# improve engine

In [35]:
df['Engine'] = df['Engine'].apply(lambda x: x.strip() if isinstance(x, str) else x)

In [36]:
body_style_counts = df['Engine'].value_counts()
valuess_to_replace = body_style_counts[body_style_counts < 22].index
df.loc[df['Engine'].isin(valuess_to_replace), 'Engine'] = np.nan

In [37]:
encoded_column = encoder.fit_transform(df[['Engine']])+1
df['Engine'] = encoded_column

In [38]:
nan_index=df[df['Engine'].isna()].index
pred_engine=df.loc[:,['Year','Make','Model','Engine']]

In [39]:
encoded_column = encoder.fit_transform(pred_engine[['Make']])
pred_engine['Make'] = encoded_column
encoded_column = encoder.fit_transform(pred_engine[['Model']])
pred_engine['Model'] = encoded_column

In [40]:
test_engin=pred_engine.loc[nan_index,['Year','Make','Model']]
train_engin=pred_engine.drop(nan_index)

In [41]:
from sklearn.ensemble import RandomForestClassifier
log_model = RandomForestClassifier()
log_model.fit(train_engin.drop('Engine',axis=1),train_engin['Engine'])
l=log_model.predict(test_engin)

In [42]:
df.loc[nan_index , 'Gearbox'] = l

# Encoding

In [43]:
# from sklearn.preprocessing import OrdinalEncoder
# encoded_column = encoder.fit_transform(df[['Make']])
# df['Make'] = encoded_column
# encoded_column = encoder.fit_transform(df[['Model']])
# df['Model'] = encoded_column
# encoded_column = encoder.fit_transform(df[['Color']])
# df['Color'] = encoded_column
# encoded_column = encoder.fit_transform(df[['Custom cleared']])
# df['Custom cleared'] = encoded_column
df=df.drop('Color',axis=1)

In [44]:
from sklearn.preprocessing import OrdinalEncoder
columns_to_encode = ['Make', 'Model','Custom cleared']
df = pd.get_dummies(df, columns=columns_to_encode, prefix=columns_to_encode)

# Drop Lines

In [45]:
# df=df.drop('Gearbox',axis=1)
# df=df.drop(columns=['Body style','Gearbox','Hand drive','Engine'],axis=1)

# prediction

In [46]:
X=df.drop('Price',axis=1)
y=df['Price']

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=51)

In [48]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler=MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [49]:
from xgboost import XGBRegressor
model=XGBRegressor(n_estimators=1000,max_depth=None,learning_rate=0.1)
model.fit(X_train,y_train)
XG_predict=model.predict(X_test)
MAE = mean_absolute_error(XG_predict,y_test)
1 - MAE / y_test.mean(),MAE

(0.7516327111816534, 3320.992321721161)

In [51]:
# df.corr()

,Year,Mileage,Body style,Gearbox,Hand drive,Engine,Price,Make_Acura,Make_Alfa,Make_Aston,...,Model_XTS,Model_XV Crosstrek,Model_Yaris,Model_Zafira,Model_eNP1,Model_i3,Model_iD.6,Model_Այլ/Օther,Custom cleared_ Մաքսազերծված,Custom cleared_ մաքսազերծված չ
0,2018,78859.465344,7.0,3.0,2.0,1.0,17500.000000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,2019,17935.000000,7.0,3.0,2.0,1.0,15500.000000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2000,269000.000000,4.0,1.0,2.0,2.0,5754.418472,False,False,False,...,False,False,False,True,False,False,False,False,True,False
3,2018,71000.000000,7.0,3.0,2.0,1.0,22000.000000,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,2020,13240.073088,7.0,3.0,2.0,1.0,9100.000000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5728,2017,18736.000000,0.0,3.0,2.0,1.0,6100.000000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5729,2020,11265.408000,0.0,3.0,2.0,1.0,6500.000000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5730,2017,19934.000000,7.0,3.0,2.0,1.0,3800.000000,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5731,2021,17935.000000,0.0,3.0,2.0,1.0,7300.000000,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [58]:
df['Custom cleared'].unique()

array([' մաքսազերծված չ', ' Մաքսազերծված', nan], dtype=object)

In [56]:
df

,Year,Make,Model,Mileage,Body style,Gearbox,Hand drive,Engine,Color,Custom cleared,Price
0,2018,BMW,745,49001 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 17 500
1,2019,Tesla,Model 3,17935 կմ,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 15 500
2,2000,Opel,Zafira,269000 կմ,Մինիվեն / Միկրոավտոբուս,Մեխանիկական,Ձախ,Գազ,Կապույտ,Մաքսազերծված,֏ 2 220 515
3,2018,Nissan,Maxima,71000 կմ,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,Մաքսազերծված,$ 22 000
4,2020,Volkswagen,Jetta,8227 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 9 100
...,...,...,...,...,...,...,...,...,...,...,...
5728,2017,Toyota,Rav 4,18736 կմ,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 6 100
5729,2020,Nissan,Rogue,7000 մղոն,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Մոխրագույն,մաքսազերծված չ,$ 6 500
5730,2017,Mazda,6,19934 կմ,Սեդան,Ավտոմատ,Ձախ,Բենզին,Արծաթագույն,մաքսազերծված չ,$ 3 800
5731,2021,Ford,Escape,17935 կմ,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,$ 7 300
